In [ ]:
import re
import xml.etree.ElementTree as ET
from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import Delaunay
from svgpathtools import svg2paths, wsvg, Line, Path as SVGPath
from choromorph import choromorph
from IPython.display import SVG
from matplotlib.patches import PathPatch
from matplotlib.path import Path as MplPath

In [ ]:

# ─────────────────────────────────────────────────────────────────────────────
# CONFIGURE YOUR INPUT / OUTPUT
# ─────────────────────────────────────────────────────────────────────────────
svg_file   = Path("../images/italy_map_grid.svg")          # original SVG (Italy outline, etc.)
output_svg = Path("../images/italy_map_grid_warped.svg")   # warped output path

# PoIs in *normalized* 0–1 space (convert later)
pois_norm = np.array([
    [0.30, 0.25],  # “pull” somewhere NW
    [0.70, 0.80],  # “pull” somewhere SE
    [0.50, 0.50],  # “pull” towards center
    [0.20, 0.70],  # “pull” somewhere NE
    [0.80, 0.30],  # “pull” somewhere SW
    [0.50, 0.20],  # “pull” somewhere S
    [0.50, 0.80],  # “pull” somewhere N
    [0.10, 0.10],  # “pull” somewhere W
    [0.90, 0.90],  # “pull” somewhere E
])

# Mesh density (40×40 is a good start for country-sized shapes)
n_x, n_y = 40, 40

# Choromorph parameters
alpha     = 0.01
beta      = 0.1
max_iter  = 60
max_step  = 5


In [ ]:

# ─────────────────────────────────────────────────────────────────────────────
# 1. Load SVG paths safely (skip shape‑to‑path conversion that may fail)
# ─────────────────────────────────────────────────────────────────────────────
paths, _ = svg2paths(
    str(svg_file),
    return_svg_attributes=False,
    convert_circles_to_paths=False,
    convert_ellipses_to_paths=False,
    convert_lines_to_paths=True,
    convert_polylines_to_paths=False,
    convert_polygons_to_paths=False,
    convert_rectangles_to_paths=False,
)
if not paths:
    raise ValueError("No <path> elements found – convert your SVG shapes to paths.")


In [ ]:

# ─────────────────────────────────────────────────────────────────────────────
# 2. Robust viewBox / size reader
# ─────────────────────────────────────────────────────────────────────────────
def _parse_viewbox_or_size(svg_path: Path):
    """Return vb_x, vb_y, vb_w, vb_h even if <svg> lacks a viewBox."""
    root = ET.parse(svg_path).getroot()

    # 1) Explicit viewBox
    if "viewBox" in root.attrib:
        return map(float, root.attrib["viewBox"].split())

    # helper: strip CSS units → float
    _to_float = lambda s: float(re.sub(r"[a-zA-Z%]+$", "", s))

    # 2) width / height attributes
    if "width" in root.attrib and "height" in root.attrib:
        w = _to_float(root.attrib["width"])
        h = _to_float(root.attrib["height"])
        return 0.0, 0.0, w, h

    # 3) Fallback – compute bbox from all paths
    pths, _ = svg2paths(str(svg_path), return_svg_attributes=False)
    if not pths:
        raise ValueError("SVG has neither viewBox, size, nor path geometry.")
    min_x = min(seg.bbox()[0] for p in pths for seg in p)
    min_y = min(seg.bbox()[1] for p in pths for seg in p)
    max_x = max(seg.bbox()[2] for p in pths for seg in p)
    max_y = max(seg.bbox()[3] for p in pths for seg in p)
    return min_x, min_y, (max_x - min_x), (max_y - min_y)


vb_x, vb_y, vb_w, vb_h = _parse_viewbox_or_size(svg_file)


In [ ]:

# ─────────────────────────────────────────────────────────────────────────────
# 3. Build regular 2‑D mesh covering the SVG viewBox
# ─────────────────────────────────────────────────────────────────────────────
xs = np.linspace(vb_x, vb_x + vb_w, n_x)
ys = np.linspace(vb_y, vb_y + vb_h, n_y)
X, Y = np.meshgrid(xs, ys)
grid = np.column_stack((X.ravel(), Y.ravel()))

# Grid connectivity (horizontal & vertical neighbours)
edges = []
for j in range(n_y):
    for i in range(n_x):
        idx = j * n_x + i
        if i < n_x - 1:  # horizontal
            edges.append((idx, idx + 1))
        if j < n_y - 1:  # vertical
            edges.append((idx, idx + n_x))
edges = np.asarray(edges, dtype=int)


In [ ]:

# ─────────────────────────────────────────────────────────────────────────────
# 4. Define PoIs in the *same coordinate system* as the mesh
# ─────────────────────────────────────────────────────────────────────────────
pois = pois_norm * np.array([vb_w, vb_h]) + np.array([vb_x, vb_y])


# ─────────────────────────────────────────────────────────────────────────────
# 5. Run Choromorph to warp the grid
# ─────────────────────────────────────────────────────────────────────────────
morphed, n_iter = choromorph(
    grid,
    pois,
    edges,
    alpha=alpha,
    beta=beta,
    max_iter=max_iter,
    max_step=max_step,
)
print(f"Choromorph converged in {n_iter} iterations")


# ─────────────────────────────────────────────────────────────────────────────
# 6. Prepare barycentric warp function via Delaunay triangulation
# ─────────────────────────────────────────────────────────────────────────────
tri = Delaunay(grid)

def barycentric_warp(pt: np.ndarray, src: np.ndarray, dst: np.ndarray, triangulation: Delaunay):
    """Warp a 2‑D point using barycentric coords between src & dst meshes."""
    simplex = triangulation.find_simplex(pt)
    if simplex == -1:  # outside the mesh
        return pt
    verts = triangulation.simplices[simplex]
    T_src = src[verts]  # (3, 2)
    T_dst = dst[verts]
    A = np.vstack((T_src.T, np.ones(3)))
    b = np.append(pt, 1)
    w = np.linalg.solve(A, b)  # barycentric weights
    return T_dst.T @ w

In [ ]:


# ─────────────────────────────────────────────────────────────────────────────
# 7. Warp every path (sample each segment, then rebuild as polylines)
# ─────────────────────────────────────────────────────────────────────────────
warped_paths = []
samples_per_seg = 30  # fidelity

for path in paths:
    new_segments = []
    for segment in path:
        t_vals = np.linspace(0, 1, samples_per_seg)
        src_pts = np.array([[segment.point(t).real, segment.point(t).imag] for t in t_vals])
        warped_pts = [barycentric_warp(pt, grid, morphed, tri) for pt in src_pts]
        for p0, p1 in zip(warped_pts[:-1], warped_pts[1:]):
            new_segments.append(Line(complex(*p0), complex(*p1)))
    warped_paths.append(SVGPath(*new_segments))


In [ ]:

# ─────────────────────────────────────────────────────────────────────────────
# VISUAL CHECK – SVG outline + ORIGINAL grid + PoIs
# ─────────────────────────────────────────────────────────────────────────────
svg_paths, _ = svg2paths(str(svg_file), return_svg_attributes=False,
                         convert_circles_to_paths=False,
                         convert_ellipses_to_paths=False,
                         convert_lines_to_paths=False,
                         convert_polylines_to_paths=False,
                         convert_polygons_to_paths=False,
                         convert_rectangles_to_paths=False)

def svgpath_to_mplpath(svg_path, samples=300):
    """Convert an svgpathtools Path to a matplotlib Path.
    Ensures that each SVG sub‑path starts with a MOVETO so we don’t draw
    spurious connecting lines between disjoint segments (which was the
    cause of the stray arrows / spikes you saw)."""
    verts, codes = [], []
    first_segment = True
    for segment in svg_path:
        ts = np.linspace(0, 1, samples)
        pts = [segment.point(t) for t in ts]
        # start a new sub‑path
        verts.append((pts[0].real, pts[0].imag))
        codes.append(MplPath.MOVETO)
        # continue the segment with straight lines between samples
        for p in pts[1:]:
            verts.append((p.real, p.imag))
            codes.append(MplPath.LINETO)
        first_segment = False
    return MplPath(verts, codes)

fig, ax = plt.subplots(figsize=(7, 7))

# SVG outline
for sp in svg_paths:
    patch = PathPatch(svgpath_to_mplpath(sp), facecolor="none", edgecolor="black", lw=1)
    ax.add_patch(patch)

# Original grid
for a, b in edges:
    ax.plot([grid[a, 0], grid[b, 0]], [grid[a, 1], grid[b, 1]], color="lightgray", lw=0.4)

# Points of Interest
ax.scatter(pois[:, 0], pois[:, 1], marker="x", s=100, color="red", label="PoIs")

ax.set_aspect("equal")
ax.set_xlim(vb_x, vb_x + vb_w)
ax.set_ylim(vb_y, vb_y + vb_h)
ax.set_title("SVG outline with grid + PoIs")
ax.legend(loc="upper right", fontsize="small")
# Flip Y‑axis so SVG (origin top‑left) aligns with Matplotlib plot
ax.set_ylim(vb_y + vb_h, vb_y)  # invert y
plt.tight_layout()
plt.tight_layout()
plt.show()

In [ ]:

# ─────────────────────────────────────────────────────────────────────────────
# VISUAL CHECK – SVG outline + MORPHED grid + PoIs
# ─────────────────────────────────────────────────────────────────────────────
fig2, ax2 = plt.subplots(figsize=(7, 7))

# Morphed grid
for a, b in edges:
    ax2.plot([morphed[a, 0], morphed[b, 0]], [morphed[a, 1], morphed[b, 1]],
             color="steelblue", lw=0.4)

# Points of Interest
ax2.scatter(pois[:, 0], pois[:, 1], marker="x", s=100, color="red", label="PoIs")

ax2.set_aspect("equal")
ax2.set_xlim(vb_x, vb_x + vb_w)
ax2.set_ylim(vb_y, vb_y + vb_h)
ax2.set_title("SVG outline with MORPHED grid + PoIs")
ax2.legend()
# Flip Y‑axis to match SVG coordinate space
ax2.set_ylim(vb_y + vb_h, vb_y)
plt.tight_layout()
plt.tight_layout()
plt.show()


In [ ]:

# ─────────────────────────────────────────────────────────────────────────────
# 8. Save & show the warped SVG
# ─────────────────────────────────────────────────────────────────────────────
wsvg(warped_paths, filename=str(output_svg))
print(f"Warped SVG saved to → {output_svg}")

# Display inline if in Jupyter
try:
    display(SVG(filename=str(output_svg)))
except NameError:
    pass
